In [1]:
import pandas as pd
import categories.commodities as coms
import categories.inflation_and_prices as ip
import utils
from functools import reduce
from dotenv import load_dotenv
import os
from fredapi import Fred

load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)

## Bacon

In [2]:
bacon_df = coms._fetch_bacon_sliced_prices()
bacon_df.head()

,Date,Bacon 1lb
0,1980-01-01,1.45
1,1980-02-01,1.41
2,1980-03-01,1.36
3,1980-04-01,1.32
4,1980-05-01,1.27


## Eggs

In [3]:
eggs_df = coms._fetch_egg_prices()
eggs_df.head()

,Date,Eggs Per Dozen
0,1980-01-01,0.879
1,1980-02-01,0.774
2,1980-03-01,0.812
3,1980-04-01,0.797
4,1980-05-01,0.737


## Milk

In [4]:
milk_df = coms._fetch_milk_prices()
milk_df.head()

,Date,Milk Per Gallon
0,1995-07-01,2.477
1,1995-08-01,2.482
2,1995-09-01,2.459
3,1995-10-01,2.473
4,1995-11-01,2.493


## Bread

In [5]:
bread_df = coms._fetch_bread_prices()
bread_df.head()

,Date,Bread 1lb
0,1980-01-01,0.501
1,1980-02-01,0.507
2,1980-03-01,0.502
3,1980-04-01,0.507
4,1980-05-01,0.504


In [6]:
eb_df = utils.merge_on_date([eggs_df, bread_df])
eb_df.head()

,Date,Eggs Per Dozen,Bread 1lb
0,1980-01-01,0.879,0.501
1,1980-02-01,0.774,0.507
2,1980-03-01,0.812,0.502
3,1980-04-01,0.797,0.507
4,1980-05-01,0.737,0.504


## Ground Beef

In [7]:
ground_beef_df = coms._fetch_ground_beef_prices()
ground_beef_df.head()

,Date,Ground Beef 1lb
0,1984-01-01,1.290
1,1984-02-01,1.340
2,1984-03-01,1.308
3,1984-04-01,1.331
4,1984-05-01,1.301


## Chicken

In [8]:
chicken_df = coms._fetch_chicken_prices()
chicken_df.head()

,Date,Chicken 1lb
0,2006-01-01,3.307
1,2006-02-01,3.341
2,2006-03-01,3.295
3,2006-04-01,3.308
4,2006-05-01,3.236


## Coffee

In [9]:
coffee_df = coms._fetch_coffee_prices()
coffee_df.head()

,Date,Coffee 1lb
0,1980-01-01,3.21
1,1980-02-01,3.26
2,1980-03-01,3.25
3,1980-04-01,3.21
4,1980-05-01,3.20


## Gas

In [10]:
gas_df = coms._fetch_gas_prices()
gas_df.head()

,Date,Gas Per Gallon
0,1976-01-01,0.605
1,1976-02-01,0.600
2,1976-03-01,0.594
3,1976-04-01,0.592
4,1976-05-01,0.600


## Electricity

In [11]:
electric_df = coms._fetch_electric_prices()
electric_df.head()

,Date,Electric Per kWh
0,1978-11-01,0.05
1,1978-12-01,0.05
2,1979-01-01,0.05
3,1979-02-01,0.05
4,1979-03-01,0.05


## CPI Dataframe

In [12]:
cpi_df = ip._fetch_cpi()
cpi_df.tail()

,Date,CPI
938,2025-03-01,319.615
939,2025-04-01,320.321
940,2025-05-01,320.580
941,2025-06-01,321.500
942,2025-07-01,322.132


## Used Autos

In [23]:
def get_used_auto_prices(start_date:str=None, end_date:str=None):
    ref_auto_cpi = 181.446
    ref_price = 28472

    # Used Auto CPI
    used_auto_series = fred.get_series('CUSR0000SETA02')
    used_auto_df = used_auto_series.to_frame().reset_index()
    used_auto_df.columns = ['Date', 'Used Auto CPI']

    # CPI
    _cpi_series = fred.get_series('CPIAUCSL')
    _cpi_df = _cpi_series.to_frame().reset_index()
    _cpi_df.columns = ['Date', 'CPI']

    used_merged = used_auto_df.merge(_cpi_df, 'inner', 'Date')

    used_merged['Used Auto Price Real'] = round(used_merged['Used Auto CPI'] * (ref_price / ref_auto_cpi),2)
    ref_cpi = used_merged['CPI'].iloc[-1]
    used_merged['Used Auto Price Nominal'] = round(used_merged['Used Auto Price Real'] * (used_merged['CPI'] / ref_cpi), 2)

    if start_date is not None:
        used_merged = used_merged[used_merged['Date'] >= start_date]
    if end_date is not None:
        used_merged = used_merged[used_merged['Date'] <= end_date]

    drop_cols = ['CPI']
    return used_merged.drop(columns=drop_cols)

used_auto_df = get_used_auto_prices(start_date="2010-01-01")
#cpi_df.tail()
used_auto_df.tail(24)

,Date,Used Auto CPI,Used Auto Price Real,Used Auto Price Nominal
847,2023-08-01,191.078,29983.43,28494.74
848,2023-09-01,187.650,29445.51,28096.51
849,2023-10-01,186.879,29324.53,28006.47
850,2023-11-01,189.444,29727.02,28430.92
851,2023-12-01,190.570,29903.71,28660.06
852,2024-01-01,183.526,28798.39,27695.38
853,2024-02-01,185.660,29133.25,28128.47
854,2024-03-01,184.709,28984.02,28082.01
855,2024-04-01,182.026,28563.01,27754.71
856,2024-05-01,180.533,28328.73,27537.96


## New Autos

In [22]:
def get_new_auto_prices(start_date:str=None, end_date:str=None):
    ref_auto_cpi = 177.552
    ref_price = 48397

    new_auto_series = fred.get_series('CUUR0000SETA01')
    new_auto_df = new_auto_series.to_frame().reset_index()
    new_auto_df.columns = ['Date', 'New Auto CPI']

    _cpi_series = fred.get_series('CPIAUCSL')
    _cpi_df = _cpi_series.to_frame().reset_index()
    _cpi_df.columns = ['Date', 'CPI']

    new_merged = new_auto_df.merge(_cpi_df, 'inner', 'Date')
    new_merged['New Auto Price Real'] = round(new_merged['New Auto CPI'] * (ref_price / ref_auto_cpi),2)
    ref_cpi = new_merged['CPI'].iloc[-1]
    new_merged['New Auto Price Nominal'] = round(new_merged['New Auto Price Real'] * (new_merged['CPI'] / ref_cpi), 2)

    if start_date is not None:
        new_merged = new_merged[new_merged['Date'] >= start_date]
    if end_date is not None:
        new_merged = new_merged[new_merged['Date'] <= end_date]

    drop_cols = ['CPI']
    
    return new_merged.drop(columns=drop_cols)

new_auto_df = get_new_auto_prices(start_date='2020-01-01')
new_auto_df.tail(24)

,Date,New Auto CPI,New Auto Price Real,New Auto Price Nominal
917,2023-08-01,179.691,48980.05,46548.17
918,2023-09-01,179.750,48996.13,46751.44
919,2023-10-01,179.446,48913.27,46714.75
920,2023-11-01,178.700,48709.92,46586.16
921,2023-12-01,178.269,48592.44,46571.55
922,2024-01-01,178.595,48681.30,46816.75
923,2024-02-01,178.592,48680.48,47001.54
924,2024-03-01,178.247,48586.44,47074.39
925,2024-04-01,178.250,48587.26,47212.29
926,2024-05-01,177.958,48507.67,47153.63


## Merged Datasets

In [15]:
from functools import reduce
import pandas as pd

dfs = [cpi_df, bacon_df, eggs_df, milk_df, bread_df, ground_beef_df, coffee_df, gas_df, electric_df]

def merge_on_date(dfs, how='inner'):
    cleaned = []
    for i, df in enumerate(dfs):
        if 'Date' not in df.columns:
            raise ValueError(f"DataFrame at index {i} is missing 'Date' column.")

        # Drop Year/Month/Day if present
        drop_cols = [c for c in ['Year', 'Month', 'Day'] if c in df.columns]
        df = df.drop(columns=drop_cols)

        cleaned.append(df)

    merged_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how=how), cleaned)
    return merged_df


def add_real_prices(df):
    latest_cpi = df['CPI'].iloc[-1]
    commodity_cols = [col for col in df.columns if col not in ["Date", "CPI"]]

    # scale each nominal price into real 2025 dollars
    for col in commodity_cols:
        df[f"{col} (Real)"] = round((df[col] * (latest_cpi / df["CPI"])),2)
    
    return df


df = merge_on_date(dfs=dfs)
df = add_real_prices(df)
df.head()


,Date,CPI,Bacon 1lb,Eggs Per Dozen,Milk Per Gallon,Bread 1lb,Ground Beef 1lb,Coffee 1lb,Gas Per Gallon,Electric Per kWh,Bacon 1lb (Real),Eggs Per Dozen (Real),Milk Per Gallon (Real),Bread 1lb (Real),Ground Beef 1lb (Real),Coffee 1lb (Real),Gas Per Gallon (Real),Electric Per kWh (Real)
0,1995-07-01,152.6,1.91,0.879,2.477,0.789,1.365,4.03,1.195,0.10,4.03,1.86,5.23,1.67,2.88,8.51,2.52,0.21
1,1995-08-01,152.9,1.97,0.984,2.482,0.797,1.328,4.05,1.164,0.10,4.15,2.07,5.23,1.68,2.80,8.53,2.45,0.21
2,1995-09-01,153.1,2.04,0.956,2.459,0.808,1.376,4.00,1.148,0.10,4.29,2.01,5.17,1.70,2.90,8.42,2.42,0.21
3,1995-10-01,153.5,2.12,0.981,2.473,0.809,1.371,3.86,1.127,0.09,4.45,2.06,5.19,1.70,2.88,8.10,2.37,0.19
4,1995-11-01,153.7,2.15,1.037,2.493,0.821,1.368,3.81,1.101,0.09,4.51,2.17,5.22,1.72,2.87,7.99,2.31,0.19


In [16]:
df = coms._fetch_all_commodity_prices()
df.tail()

,Date,Bacon 1lb,Bacon 1lb (Real),Eggs Per Dozen,Eggs Per Dozen (Real),Milk Per Gallon,Milk Per Gallon (Real),Bread 1lb,Bread 1lb (Real),Ground Beef 1lb,Ground Beef 1lb (Real),Coffee 1lb,Coffee 1lb (Real),Gas Per Gallon,Gas Per Gallon (Real),Electric Per kWh,Electric Per kWh (Real)
356,2025-03-01,6.98,7.03,6.227,6.28,4.050,4.08,1.880,1.89,5.790,5.84,7.38,7.44,3.232,3.26,0.18,0.18
357,2025-04-01,7.01,7.05,5.122,5.15,4.074,4.10,1.913,1.92,5.801,5.83,7.54,7.58,3.330,3.35,0.18,0.18
358,2025-05-01,6.98,7.01,4.548,4.57,4.022,4.04,1.876,1.89,5.981,6.01,7.93,7.97,3.306,3.32,0.18,0.18
359,2025-06-01,7.10,7.11,3.775,3.78,4.029,4.04,1.864,1.87,6.120,6.13,8.13,8.15,3.306,3.31,0.19,0.19
360,2025-07-01,7.12,7.12,3.599,3.60,4.162,4.16,1.851,1.85,6.254,6.25,8.41,8.41,3.285,3.28,0.19,0.19
